In [1]:
#from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import warnings
import numpy as np
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
train_data = pd.read_csv('adult.data',header = None)
train_data.columns = {'age':int,
               'workclass':'category',
               'fnlwgt':int,
               'education':'category',
                'education_num':int,
               'marital-status':'category',
               'occupation':'category',
               'relationship':'category',
               'race':'category',
               'sex':'category',
               'capital-gain':int,
               'capital-loss':int,
               'hours-per-week':int,
               'native-country':'category',
               'listing_of_attributes':'category'}
train_data

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,listing_of_attributes
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
test_data = pd.read_csv('adult.test',header = None)
test_data.columns = {'age':int,
               'workclass':'category',
               'fnlwgt':int,
               'education':'category',
                'education_num':int,
               'marital-status':'category',
               'occupation':'category',
               'relationship':'category',
               'race':'category',
               'sex':'category',
               'capital-gain':int,
               'capital-loss':int,
               'hours-per-week':int,
               'native-country':'category',
               'listing_of_attributes':'category'}
test_data

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,listing_of_attributes
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [4]:
train_data.apply(lambda x:np.sum(x==' ?'))

age                         0
workclass                1836
fnlwgt                      0
education                   0
education_num               0
marital-status              0
occupation               1843
relationship                0
race                        0
sex                         0
capital-gain                0
capital-loss                0
hours-per-week              0
native-country            583
listing_of_attributes       0
dtype: int64

In [5]:
test_data.apply(lambda x:np.sum(x==' ?'))

age                        0
workclass                963
fnlwgt                     0
education                  0
education_num              0
marital-status             0
occupation               966
relationship               0
race                       0
sex                        0
capital-gain               0
capital-loss               0
hours-per-week             0
native-country           274
listing_of_attributes      0
dtype: int64

In [6]:
train_data.drop('fnlwgt', axis= 1,inplace= True)
#train_data.drop('listing_of_attributes', axis= 1, inplace= True)
######################################################################
test_data.drop('fnlwgt', axis= 1,inplace= True)
#test_data.drop('listing_of_attributes', axis= 1, inplace= True)

In [7]:
#查詢缺失值，並丟棄不需要的屬性欄位
train_data.replace(" ?", np.nan, inplace= True)
new_train_data = train_data.dropna()
le=LabelEncoder()
for col in new_train_data[['workclass','education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']]:
    new_train_data[col]=le.fit_transform(new_train_data[col])
print(r'總比數紀錄:{}'.format(train_data.shape[0]))
print(r'刪除含缺漏紀錄資料後比數:{}'.format(train_data.dropna().shape[0]))
pd.get_dummies(new_train_data['workclass'])
workclass_encoding = pd.get_dummies(new_train_data['workclass'], prefix= 'workclass')

pd.get_dummies(new_train_data['education'])
educations_encoding = pd.get_dummies(new_train_data['education'], prefix= 'education')
new_train_data = new_train_data.drop('education', 1)

pd.get_dummies(new_train_data['marital-status'])
marital_status_encoding = pd.get_dummies(new_train_data['marital-status'], prefix= 'marital-status')
#new_train_data = new_train_data.drop('marital-status', 1)

pd.get_dummies(new_train_data['occupation'])
occupation_encoding = pd.get_dummies(new_train_data['occupation'], prefix= 'occupation')
#pd.DataFrame(occupation_encoding)

pd.get_dummies(new_train_data['relationship'])
relationship_encoding = pd.get_dummies(new_train_data['relationship'], prefix= 'relationship')
new_train_data = new_train_data.drop('relationship', 1)

pd.get_dummies(new_train_data['race'])
race_encoding = pd.get_dummies(new_train_data['race'], prefix= 'race')

pd.get_dummies(new_train_data['sex'])
sex_encoding = pd.get_dummies(new_train_data['sex'], prefix= 'sex')

pd.get_dummies(new_train_data['native-country'])
native_country_encoding = pd.get_dummies(new_train_data['native-country'], prefix= 'native-country')

pd.get_dummies(new_train_data['listing_of_attributes'])
listing_of_attributes_encoding = pd.get_dummies(new_train_data['listing_of_attributes'], prefix= 'listing_of_attributes')

#class_mapping = {' >50K': 1, ' <=50K': 0}
#new_train_data['listing_of_attributes'] = new_train_data['listing_of_attributes'].map(class_mapping)

pd.DataFrame(new_train_data)

總比數紀錄:32561
刪除含缺漏紀錄資料後比數:30162


,age,workclass,education_num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,listing_of_attributes
0,39,5,13,4,0,4,1,2174,0,40,38,<=50K
1,50,4,13,2,3,4,1,0,0,13,38,<=50K
2,38,2,9,0,5,4,1,0,0,40,38,<=50K
3,53,2,7,2,5,2,1,0,0,40,38,<=50K
4,28,2,13,2,9,2,0,0,0,40,4,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,2,12,2,12,4,0,0,0,38,38,<=50K
32557,40,2,9,2,6,4,1,0,0,40,38,>50K
32558,58,2,9,6,0,4,0,0,0,40,38,<=50K
32559,22,2,9,4,0,4,1,0,0,20,38,<=50K


In [8]:
#合併 onehot encoding
new_train_data = pd.concat([workclass_encoding,educations_encoding,marital_status_encoding,occupation_encoding,
           relationship_encoding,race_encoding,sex_encoding, listing_of_attributes_encoding, new_train_data['age'],new_train_data['education_num'],new_train_data['capital-gain'],new_train_data['capital-loss'],new_train_data['hours-per-week']],axis=1)
new_train_data

,workclass_0,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,education_0,education_1,education_2,...,race_4,sex_0,sex_1,listing_of_attributes_ <=50K,listing_of_attributes_ >50K,age,education_num,capital-gain,capital-loss,hours-per-week
0,0,0,0,0,0,1,0,0,0,0,...,1,0,1,1,0,39,13,2174,0,40
1,0,0,0,0,1,0,0,0,0,0,...,1,0,1,1,0,50,13,0,0,13
2,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,38,9,0,0,40
3,0,0,1,0,0,0,0,0,1,0,...,0,0,1,1,0,53,7,0,0,40
4,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,28,13,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,0,1,0,0,0,0,0,0,0,...,1,1,0,1,0,27,12,0,0,38
32557,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,40,9,0,0,40
32558,0,0,1,0,0,0,0,0,0,0,...,1,1,0,1,0,58,9,0,0,40
32559,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,22,9,0,0,20


In [9]:
test_data.replace(" ?", np.nan, inplace= True)
new_test_data = test_data.dropna()
le=LabelEncoder()
for col in new_test_data[['workclass','education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']]:
    new_test_data[col]=le.fit_transform(new_test_data[col])
print(r'總比數紀錄:{}'.format(test_data.shape[0]))
print(r'刪除含缺漏紀錄資料後比數:{}'.format(test_data.dropna().shape[0]))
pd.get_dummies(new_test_data['workclass'])
workclass_encoding = pd.get_dummies(new_test_data['workclass'], prefix= 'workclass')

pd.get_dummies(new_test_data['education'])
educations_encoding = pd.get_dummies(new_test_data['education'], prefix= 'education')
new_test_data = new_test_data.drop('education', 1)

pd.get_dummies(new_test_data['marital-status'])
marital_status_encoding = pd.get_dummies(new_test_data['marital-status'], prefix= 'marital-status')
#new_test_data = new_test_data.drop('marital-status', 1)

pd.get_dummies(new_test_data['occupation'])
occupation_encoding = pd.get_dummies(new_test_data['occupation'], prefix= 'occupation')

pd.get_dummies(new_test_data['relationship'])
relationship_encoding = pd.get_dummies(new_test_data['relationship'], prefix= 'relationship')
new_test_data = new_test_data.drop('relationship', 1)

pd.get_dummies(new_test_data['race'])
race_encoding = pd.get_dummies(new_test_data['race'], prefix= 'race')

pd.get_dummies(new_test_data['sex'])
sex_encoding = pd.get_dummies(new_test_data['sex'], prefix= 'sex')

pd.get_dummies(new_test_data['native-country'])
native_country_encoding = pd.get_dummies(new_test_data['native-country'], prefix= 'native-country')

pd.get_dummies(new_test_data['listing_of_attributes'])
listing_of_attributes_encoding = pd.get_dummies(new_test_data['listing_of_attributes'], prefix= 'listing_of_attributes')

#class_mapping = {' >50K': 1, ' <=50K': 0}
#new_test_data['listing_of_attributes'] = new_test_data['listing_of_attributes'].map(class_mapping)

pd.DataFrame(new_test_data)

總比數紀錄:16281
刪除含缺漏紀錄資料後比數:15060


,age,workclass,education_num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,listing_of_attributes
0,25,2,7,4,6,2,1,0,0,40,37,<=50K.
1,38,2,9,2,4,4,1,0,0,50,37,<=50K.
2,28,1,12,2,10,4,1,0,0,40,37,>50K.
3,44,2,10,2,6,2,1,7688,0,40,37,>50K.
5,34,2,6,4,7,4,1,0,0,30,37,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...
16275,33,2,13,4,9,4,1,0,0,40,37,<=50K.
16276,39,2,13,0,9,4,0,0,0,36,37,<=50K.
16278,38,2,13,2,9,4,1,0,0,50,37,<=50K.
16279,44,2,13,0,0,1,1,5455,0,40,37,<=50K.


In [10]:
#合併 onehot encoding
new_test_data = pd.concat([workclass_encoding,educations_encoding,marital_status_encoding,occupation_encoding,
           relationship_encoding,race_encoding,sex_encoding, listing_of_attributes_encoding, new_test_data['age'],new_test_data['education_num'],new_test_data['capital-gain'],new_test_data['capital-loss'],new_test_data['hours-per-week']],axis=1)
new_test_data

,workclass_0,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,education_0,education_1,education_2,...,race_4,sex_0,sex_1,listing_of_attributes_ <=50K.,listing_of_attributes_ >50K.,age,education_num,capital-gain,capital-loss,hours-per-week
0,0,0,1,0,0,0,0,0,1,0,...,0,0,1,1,0,25,7,0,0,40
1,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,38,9,0,0,50
2,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,28,12,0,0,40
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,44,10,7688,0,40
5,0,0,1,0,0,0,0,1,0,0,...,1,0,1,1,0,34,6,0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16275,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,33,13,0,0,40
16276,0,0,1,0,0,0,0,0,0,0,...,1,1,0,1,0,39,13,0,0,36
16278,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,38,13,0,0,50
16279,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,44,13,5455,0,40


In [11]:
new_cols_train=list(new_train_data.columns)
new_cols_train.remove('hours-per-week')

new_cols_test=list(new_test_data.columns)
new_cols_test.remove('hours-per-week')

x_train, y_train = new_train_data[new_cols_train].values, new_train_data['hours-per-week'].values
x_test, y_test = new_test_data[new_cols_test].values, new_test_data['hours-per-week'].values

In [12]:
#enc= OneHotEncoder(handle_unknown='ignore')
#enc = enc.fit(x_train)
#X_train = enc.transform(x_train)
#X_test = enc.transform(x_test)

In [13]:
#KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 9.65179282868526
RMSE : 14.44
MAPE: 0.33168798823766604


In [14]:
#RandomForest
# 建立 Random Forest Classifier 模型
randomForestModel = RandomForestClassifier(n_estimators=100, criterion = 'gini')
randomForestModel.fit(x_train, y_train)
y_pred_rf = randomForestModel.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_rf))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_rf,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_rf))

MAE: 8.420517928286852
RMSE : 13.31
MAPE: 0.3016668731860754


In [15]:
#XGBoost
xgbc = XGBRegressor()
xgbc.fit(x_train, y_train)
y_pred_xgb = xgbc.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_xgb))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_xgb,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_xgb))

MAE: 7.107676320183641
RMSE : 10.46
MAPE: 0.2622871460866288


In [16]:
#SVR
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
regr.fit(x_train, y_train)
y_pred_lsvr = regr.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_lsvr))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_lsvr,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_lsvr))

MAE: 7.114624819559816
RMSE : 11.63
MAPE: 0.2946015257793725
